In [1]:
import os
import pyodbc
from os.path import join
from datetime import datetime
from RetrieveRetrosheet import Retrosheet



In [2]:
def create_connection(driver, server, db, tc = 'yes'):
    conn = pyodbc.connect("""Driver={};
                      Server={};
                      Database={};
                      Trusted_Connection={};""".format(driver,server,db,tc))
    
    return conn

In [3]:
conn = create_connection('SQL Server', {Server}, {Database})



In [4]:
def import_play(startyear, endyear = None, path = 'C:\\retro\\RawFiles\\csvs', db = 'Retrosheet'):
    
    cursor = conn.cursor()
    now = datetime.now()

    if isinstance(startyear, int):
        
        if not isinstance(endyear, int):
            endyear = startyear
            
        if endyear < startyear:
            endyear = startyear
        
        logfile = open('C:\\retro\\ProcessLog\\log_{}{}{}.txt'.format(now.year, now.month, now.day), 'a')
        
        try:
            logfile.write('\n\nStarting Additional Processing for import_play as of {}.\n\n'.format(now))
            
        except OSError:
            logfile = open('C:\\retro\\ProcessLog\\log_{}{}{}.txt'.format(now.year, now.month, now.day), 'w')
            logfile.write('Starting new file as of {}\n\nStarting import_play:'.format(now))
        
        for (dirname, dirs, files) in os.walk(path):
            i = 1
            oldcount = int(cursor.execute('Select Count(*) From stage.PlayByPlay').fetchval())
            for filename in files:
                
                f = join(path, filename)
                
                if int(filename[0:4]) >= startyear and int(filename[0:4]) <= endyear:
                    
                    sqlcmd = '''Bulk insert stage.PlayByPlay
                                From ''' + '\'' + f + '\'' + \
                                '''\nWITH
                                (
                                Firstrow = 1,
                                FieldTerminator = ',',
                                RowTerminator = '\n',
                                Format = 'CSV'
                                )'''
                    logfile.write('\n\nProcessing: {}'.format(f))
                    #print(sqlcmd)
                    try:
                        cursor.execute(sqlcmd)
                        cursor.commit()
                        success = 1
                    except: 
                        logfile.write('\n\nERROR--Could not execute: {}\n\n'.format(sqlcmd))
                        success = 0
                    
                    newcount = int(cursor.execute('Select Count(*) From stage.PlayByPlay').fetchval())
                    diff = newcount-oldcount
                    logcmd = """Insert Into ETL_Processing.dbo.FileImport 
                                (ImportedFile, RowsInserted, InsertTime, DestTable, IsSuccessful, db) 
                                \nValues('{}', {}, GetDate(), 'stage.PlayByPlay', {}, '{}')""".format(f, diff, success, db)
                    #+ f + '\', ' + str(diff) + ', GetDate())'
                    logfile.write('\n\n{}'.format(logcmd))
                    cursor.execute(logcmd)
                    cursor.commit()
                    logfile.write('\n\nRows inserted into stage.PlayByPlay: {}'.format(str(diff)))
                    logfile.write('\n\nDone with file: {}'.format(f))
                    
                    oldcount += diff
                    i +=1
                
                else:
                    logfile.write('\n\nSkipped: {}\n\tNo rows were processed'.format(f))
                #if i >=1:
                #    break
        
    else:
        print('Startyear must be an integer value')
    
    logfile.close()
    cursor.close()

In [5]:
import_play(1998, 2000, db = 'Retrodev')

In [50]:
conn.close()